# 機械学習入門

*機械学習入門*と呼ばれる記事は、おそらくwebサイト上に数千件あります。本稿では、これらの記事を書き直すのではなく、化学での応用に焦点を当てた主要な概念を紹介することにします。 以下に入門的な資料を挙げます。新しくてより良いものがあれば教えてください。

1. 原著者が大学院で初めて読んだ、機械学習についてのEthem Alpaydinによって書かれた本{cite}`alpaydin2020introduction`
2. Nils Nillsonのオンラインブック [<ins>Introductory Machine Learning</ins>](https://ai.stanford.edu/~nilsson/mlbook.html)
3. 物質科学における機械学習についての2つのreview{cite}`fung2021benchmarking,balachandran2019machine`
4. 計算科学における機械学習についての2つのreview{cite}`gomez2020machine`
5. 金属科学における機械学習についての2つのreview{cite}`nandy2018strategies`

これらの資料から、機械学習がデータをモデリングする手法であり、一般的にはには予測機能を持つことを学んでいただけると思います。機械学習には多くの手法が含まれますが、ここでは深層学習を学ぶために必要なものだけを取り上げます。例えば、ランダムフォレスト、サポートベクターマシン、最近傍探索などは広く使われている機械学習手法で、今でも有効な手法ですが、ここでは取り上げません。

```{admonition} 読者層と目的
本章は、化学とpythonについてある程度知識のある機械学習の初心者を対象としており、そうでない場合には上記の入門記事のいずれかに目を通しておくことをお勧めします。この記事では、`pandas`の知識（カラムの読み込みと選択）、`rdkit`の知識（分子の描き方）、分子を[SMILES](https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system) {cite}`weininger1988smiles`として保存する方法についてある程度の知識を想定しています。この章を読むと、以下のことができるようになると想定されます。

  * 特徴量、ラベルの定義
  * 教師あり学習と教師なし学習を区別する。
  * 損失関数とは何か、勾配降下法を用いてどのように最小化できるかを理解する。
  * モデルとは何か、特徴とラベルとの関係を理解する。
  * データのクラスタリングができ、それがデータについて何を示すかを説明できる。
```

## The Ingredients 

機械学習とは、データに当てはめてモデルを構築することを目指す分野です。
まず、言葉を定義します。

**特徴量** 

&nbsp;&nbsp;&nbsp;&nbsp;次元$D$の$N$個のベクトル $\{\vec{x}_i\}$ の集合です。実数、整数等が用いられます。

**ラベル** 

&nbsp;&nbsp;&nbsp;&nbsp;$N$ 個の整数または実数の集合 $\{y_i\}$。$y_i$ は通常スカラーです。
  
**ラベル付きデータ** 

&nbsp;&nbsp;&nbsp;&nbsp;$N$ 個のtupleからなる集合 $\{\left(\vec{x}_i, y_i\right)\}$ を指します。

**ラベルなしデータ** 

&nbsp;&nbsp;&nbsp;&nbsp;ラベル $y$ が未知の $N$ 個の特徴量 $\{\vec{x}_i\}$ の集合を指します。

**モデル**

&nbsp;&nbsp;&nbsp;&nbsp;特徴ベクトルを受け取り、予測結果 $\hat{y}$ を出力する関数 $f(\vec{x})$ を指します。

**予測結果**

&nbsp;&nbsp;&nbsp;&nbsp; 与えられた入力 $\vec{x}$ に対し、モデルを通して得られた予測結果 $\hat{y}$ のことを指します。

## 教師あり学習

最初のタスクは**教師あり学習**です。教師あり学習とは、データで学習したモデルで $\vec{x}$ から $y$ を予測する方法です。このタスクは、データセットに含まれるラベルをアルゴリズムに教えることで学習を進めるため、*教師あり*学習と呼ばれています。もう一つの方法は**教師なし学習**で、アルゴリズムにラベルを教えない方法です。この教師あり／教師なしの区別は後でもっと厳密になりますが、今のところはこの定義で十分です。

例として、AqSolDB{cite}`Sorkun2019`という、約1万種類の化合物と、その水への溶解度の測定結果(ラベル)についてのデータセットを使ってみます。このデータセットには、機械学習に利用できる分子特性（特徴量）も含まれています。溶解度の測定結果は、化合物の水への溶解度をlog molarityの単位で表したものになります。

## Running This Notebook


上にある &nbsp;<i aria-label="Launch interactive content" class="fas fa-rocket"></i>&nbsp; をクリックすると、このページがインタラクティブなGoogle Colab Notebookとして起動されるようになります。 パッケージのインストールについては、以下を参照してください。

````{tip} My title
:class: dropdown
パッケージをインストールするには、新しいセルで次のコードを実行します。

```
!pip install dmol-book
```

インストールに問題が生じた場合は、[このリンク](https://github.com/whitead/dmol-book/blob/master/package/requirements.txt)から使用されているパッケージリストの最新版を入手することができます。

````

### データのロード

データをダウンロードし、[Pandas](https://pandas.pydata.org/)のデータフレームにロードします。以下のセルでは、インポートや必要なパッケージのインストールを設定します。

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import jax.numpy as jnp
import jax
from jax.example_libraries import optimizers
import sklearn.manifold, sklearn.cluster
import rdkit, rdkit.Chem, rdkit.Chem.Draw
import dmol

In [ ]:
# soldata = pd.read_csv('https://dataverse.harvard.edu/api/access/datafile/3407241?format=original&gbrecs=true')
# had to rehost because dataverse isn't reliable
soldata = pd.read_csv(
    "https://github.com/whitead/dmol-book/raw/master/data/curated-solubility-dataset.csv"
)
soldata.head()

### データ探索

```{margin} EDA
EDAを特徴量の選択として行う場合、テストデータでモデルの選択に影響を与えないように、EDAを行う前にtrain/test/(valid)の分割を行う必要があります。
```

分子には、分子量、回転可能な結合、価電子など、様々な特徴量となりうるものがあります。そしてもちろん、今回のデータセットにおいてラベルとなる**溶解度**という指標もあります。このデータセットに対して私たちが常に最初に行うべきことの一つは、**探索的データ解析**（EDA）と呼ばれるプロセスでデータについての理解を深めることです。まず、ラベルやデータの大枠を知るために、いくつかの具体的な例を調べることから始めましょう。

In [ ]:
# plot one molecule
mol = rdkit.Chem.MolFromInchi(soldata.InChI[0])
mol

これはデータセットのうち、最初の分子を[rdkit](https://rdkit.org/)を使ってレンダリングしたものです。

それでは、溶解度データの**範囲**とそれを構成する分子についてなんとなく理解するために、極値を見てみましょう。まず、溶解度の確率分布の形と極値を知るために、({obj}`seaborn.distplot` を使って)溶解度のヒストグラムを作成します。

In [ ]:
sns.distplot(soldata.Solubility)
plt.show()

上図では、溶解度のヒストグラムとカーネル密度推定値を重ね合わせています。このヒストグラムから、溶解度は約-13から2.5まで変化し、正規分布していないことがわかります。

In [ ]:
# get 3 lowest and 3 highest solubilities
soldata_sorted = soldata.sort_values("Solubility")
extremes = pd.concat([soldata_sorted[:3], soldata_sorted[-3:]])

# We need to have a list of strings for legends
legend_text = [
    f"{x.ID}: solubility = {x.Solubility:.2f}" for x in extremes.itertuples()
]

# now plot them on a grid
extreme_mols = [rdkit.Chem.MolFromInchi(inchi) for inchi in extremes.InChI]
rdkit.Chem.Draw.MolsToGridImage(
    extreme_mols, molsPerRow=3, subImgSize=(250, 250), legends=legend_text
)

極端な分子の例では、高塩素化合物が最も溶解度が低く、イオン性化合物が溶解度が高いことがわかります。A-2918は**外れ値**、つまり間違いなのでしょうか？また、NH$_3$ は本当にこれらの有機化合物に匹敵するのでしょうか？このような疑問は、モデリングを行う*前に*検討すべきことです。

```{margin} 外れ値

外れ値とは、正規のデータ分布から外れた極端な値のことです。間違いであったり、異なる分布であったりします。（例えば、有機分子ではなく金属であるなど）外れ値はモデル学習に強い影響を与える可能性があります。

```

### 特徴量の相関
次に、特徴量と溶解度(ラベル)の相関を調べてみましょう。 `SD` (標準偏差)、`Ocurrences` (その分子が構成するデータベースで何回出現したか)、`Group` (データの出所) など、特徴量や溶解度とは関係のないカラムがいくつかあることに注意してください。

In [ ]:
features_start_at = list(soldata.columns).index("MolWt")
feature_names = soldata.columns[features_start_at:]

fig, axs = plt.subplots(nrows=5, ncols=4, sharey=True, figsize=(12, 8), dpi=300)
axs = axs.flatten()  # so we don't have to slice by row and column
for i, n in enumerate(feature_names):
    ax = axs[i]
    ax.scatter(
        soldata[n], soldata.Solubility, s=6, alpha=0.4, color=f"C{i}"
    )  # add some color
    if i % 4 == 0:
        ax.set_ylabel("Solubility")
    ax.set_xlabel(n)
# hide empty subplots
for i in range(len(feature_names), len(axs)):
    fig.delaxes(axs[i])
plt.tight_layout()
plt.show()

分子量や水素結合の数は、少なくともこのプロットからは、ほとんど相関がないように見えるのは興味深いことです。MolLogPは溶解性に関連する計算から導出された記述子で、よい相関を持っています。また、これらの特徴量のいくつかは、**分散**が低く、特徴の値が多くのデータに対してほとんど変化しないか、全く変化しないことがわかります（例えば、「NumHDonors」など）。

### 線形モデル

まず、最も単純なアプローチの1つである線形モデルから始めましょう。これは教師あり学習の最初の例で、これから説明する特徴量の選択が難しいため、ほとんど使われることはありません。


```{margin} Autodiff
[Autodiff](https://en.wikipedia.org/wiki/Automatic_differentiation) は2つの変数に関する分析的な勾配を計算することができるツールです。
```

この線形モデルは以下の方程式で定義されます。

\begin{equation}
    y = \vec{w} \cdot \vec{x} + b
\end{equation}

この式は1つのデータ点に対して定義されます。1つの特徴ベクトル $\vec{x}$ の形状は、(17個の特徴があるため)今回の場合17です。$\vec{w}$ は長さ17の調整可能なパラメータのベクトルで、 $b$ は調整可能なスカラーです(**バイアス** と呼ばれます)。

このモデルは、[``jax``](https://jax.readthedocs.io/en/latest/notebooks/quickstart.html)というライブラリを用いて実装します。このライブラリは、autodiffによって解析的勾配を簡単に計算できることを除けば、numpyに非常によく似ています。

In [ ]:
def linear_model(x, w, b):
    return jnp.dot(x, w) + b


# test it out
x = np.array([1, 0, 2.5])
w = np.array([0.2, -0.5, 0.4])
b = 4.3

linear_model(x, w, b)

```{margin} 損失
損失とは、モデルの予測値 $\hat{y}$ とラベル $y$ を受け取り、モデルがどれだけ適合しているかを表す関数から出力されるスカラーです。今回の目標は、この損失を最小化することです。
```

ここで重要な問題が出てきます。 *調整可能なパラメータ $\vec{w}$ と $b$ はどのように見つけるのでしょうか？* 線形回帰の古典的な方法では、 $\vec{w} = (X^TX)^{-1}X^{T}\vec{y}$ という擬似逆行列を使って調整パラメータを直接計算します。詳しくは [こちら](https://nbviewer.jupyter.org/github/whitead/numerical_stats/blob/master/unit_12/lectures/lecture_1.ipynb#Extending-Least-Squares-to-Multiple-Dimensions-in-Domain---OLS-ND)に詳しく書いてあります。 しかし、今回は、深層学習で行うことを考慮した**反復的**なアプローチを使用します。これは線形回帰の正しい計算方法ではありませんが、深層学習ではよく見る方法なので、反復的な計算方法に慣れるのに便利でしょう。

調整可能なパラメータを繰り返し見つけるために、**損失**関数を選び、**勾配**で最小化することにします。これらの量を定義し、いくつかの初期値wとbを用いて損失を計算していきます。

In [ ]:
# convert data into features, labels
features = soldata.loc[:, feature_names].values
labels = soldata.Solubility.values

feature_dim = features.shape[1]

# initialize our paramaters
w = np.random.normal(size=feature_dim)
b = 0.0

# define loss
def loss(y, labels):
    return jnp.mean((y - labels) ** 2)


# test it out
y = linear_model(features, w, b)
loss(y, labels)

溶解度が-13から2であることを考えると、この損失はひどいものです。しかし、この結果はあくまで初期パラメータから予測しただけなので、正しいのです。


### 勾配降下法

ここで、調整可能なパラメータに対して損失がどのように変化するかという情報を使って、損失を減らしていきます。
今回用いる損失関数を以下のように定義します。:

\begin{equation}
    L = \frac{1}{N}\sum_i^N \left[y_i - f(\vec{x}_i, \vec{w}, b)\right]^2
\end{equation}

この損失関数は**平均2乗誤差**と呼ばれ、しばしばMSEと略されます。調整可能なパラメータに対する損失の勾配を計算することができます。

```{margin} jax.grad
[jax.grad](https://jax.readthedocs.io/en/latest/jax.html#jax.grad)はPython関数の解析的導関数を計算します。
これは2つの引数を取ります。関数と、どの引数に対して微分を行うかについてです。
例えば、`f(x, y, z)`について考えると、`jax.grad(f,(1,2))` は $\frac{\partial f}{\partial x}, \frac{\partial f}{\partial y}$ を与えます。 $x$ はテンソルでも良いことに注意してください。 
```

\begin{equation}
    \frac{\partial L}{\partial w_i}, \frac{\partial L}{\partial b}
\end{equation}

ここで、$w_i$ は重みベクトル $i$ 番目の要素である $\vec{w}$ です。負の勾配の方向に微小量の変化を生むことで、損失を減らすことができます。:

\begin{equation}
    (w_i, b') = \left(w_i - \eta \frac{\partial L}{\partial w_i}, b - \eta\frac{\partial L}{\partial b}\right)
\end{equation}

ここで、 $\eta$ は**学習率**であり、調整可能であるが、学習しないパラメータ（**ハイパーパラメータ**と呼ばれます）です。この例では $1\times10^{-6}$ と設定しています。一般的には10の累乗で表され、最大でも0.1程度になるように設定されます。それ以上の値は安定性に問題があることが知られています。それでは、**gradient descent**と呼ばれるこの手順を実装してみます。

In [ ]:
# compute gradients
def loss_wrapper(w, b, data):
    features = data[0]
    labels = data[1]
    y = linear_model(features, w, b)
    return loss(y, labels)


loss_grad = jax.grad(loss_wrapper, (0, 1))

# test it out
loss_grad(w, b, (features, labels))

勾配の計算を行ったので、それを数ステップかけて最小化します。

In [ ]:
loss_progress = []
eta = 1e-6
data = (features, labels)
for i in range(10):
    grad = loss_grad(w, b, data)
    w -= eta * grad[0]
    b -= eta * grad[1]
    loss_progress.append(loss_wrapper(w, b, data))
plt.plot(loss_progress)

plt.xlabel("Step")
plt.yscale("log")
plt.ylabel("Loss")
plt.title("Full Dataset Training Curve")
plt.show()

### 学習曲線

上の図は **学習曲線** と呼ばれるものです。これは損失が減少しているかどうかを示しており、モデルが学習を行っていることを表しています。学習曲線は **Learning Curve** とも呼ばれます。X軸はサンプル数、データセットの総反復回数（エポックと呼ばれる）、モデルの学習に使用されたデータ量の他の指標が用いられます。

### バッチ処理

```{margin} バッチ
バッチはデータのサイズに応じたデータの部分集合のことです。バッチサイズは通常2の累乗で定義されます。（例：4、16、128）。
ランダムなバッチを用いることで、勾配降下法は確率的勾配降下法になります。
```

勾配降下法によって学習が良い感じに進んでいることがわかります。しかし、ちょっとした変更で学習のスピードアップを図ることができます。機械学習で実際に行われている学習方法である **バッチ処理** を使ってみましょう。ちょっとした変更点とは、すべてのデータを一度に使うのではなく、その部分集合の小さな**バッチ**データだけを取るということです。バッチ処理には2つの利点があります。1つはパラメータの更新を計算する時間を短縮できること、もう1つは学習過程をランダムにできることです。このランダム性により、学習の進行を止める可能性のあるる局所的な極小値から逃れることができます。このバッチ処理の追加により、この勾配降下法アルゴリズムは**確率的**となり、**確率的勾配降下法**（SGD）と呼ばれる手法になります。SGDとそのバリエーションは、深層学習における最も一般的な学習方法です。

In [ ]:
# initialize our paramaters
# to be fair to previous method
w = np.random.normal(size=feature_dim)
b = 0.0

loss_progress = []
eta = 1e-6
batch_size = 32
N = len(labels)  # number of data points
data = (features, labels)
# compute how much data fits nicely into a batch
# and drop extra data
new_N = len(labels) // batch_size * batch_size

# the -1 means that numpy will compute
# what that dimension should be
batched_features = features[:new_N].reshape((-1, batch_size, feature_dim))
batched_labels = labels[:new_N].reshape((-1, batch_size))
# to make it random, we'll iterate over the batches randomly
indices = np.arange(new_N // batch_size)
np.random.shuffle(indices)
for i in indices:
    # choose a random set of
    # indices to slice our data
    grad = loss_grad(w, b, (batched_features[i], batched_labels[i]))
    w -= eta * grad[0]
    b -= eta * grad[1]
    # we still compute loss on whole dataset, but not every step
    if i % 10 == 0:
        loss_progress.append(loss_wrapper(w, b, data))

plt.plot(np.arange(len(loss_progress)) * 10, loss_progress)
plt.xlabel("Step")
plt.yscale("log")
plt.ylabel("Loss")
plt.title("Batched Loss Curve")
plt.show()

ここで注目すべき点は、以下の3つです。

1. バッチ処理を行わない場合に比べ、損失が小さくなっています。
2. データセットを10回で反復するのをやめ、1回だけ反復しているにも関わらず、ステップ数が増えています。
3. 損失は常に減少するわけではありません。

損失が小さくなる理由は、各データポイントを1回しか見ていないにもかかわらず、より多くのステップを踏むことができるからです。バッチ処理を行うと、バッチごとに勾配降下法の更新を行うため、データセットに対して1回の反復でより多くの更新を行うことができます。具体的には $B$ をバッチサイズとすると、元の勾配降下法では1回しか更新できない所、バッチ処理を行った場合は $N / B$ 回の更新を行うことができます。損失が常に減少しない理由は、評価するたびに異なるデータセットであるためです。データセットからランダムに選定したバッチでは、ある分子が他の分子より予測が難しかったり、1つのバッチに基づいてパラメータを更新しただけなので、各ステップが正しく損失を最小化できているとは限らなかったりします。しかし、バッチがランダムに選定されていると仮定すれば、常に（平均的に）損失の減少量の期待値を向上させることができます。(つまり、損失の期待値を最小化できます)

### 特徴量の標準化

It seems we cannot get past a certain loss. If you examine the gradients you'll see some of them are very large and some are very small. Each of the features have different magnitudes. For example, molecular weights are large numbers. The number of rings in a molecule is a small number. Each of these must use the same learning rate, $\eta$, and that is ok for some but too small for others. If we increase $\eta$, our training procedure will explode because of these larger feature gradients. A standard trick we can do is make all the features have the same magnitude, using the equation for standardization you might see in your statistics textbook:

\begin{equation}
    x_{ij} = \frac{x_{ij} - \bar{x_j}}{\sigma_{x_j}}
\end{equation}

where $\bar{x_j}$ is column mean and $\sigma_{x_j}$ is column standard deviation. To be careful about contaminating training data with test data -- leaking information between train and test data -- we should only use training data in computing the mean and standard deviation. We want our test data to approximate how we'll use our model on unseen data, so we cannot know what these unseen features means/standard deviations might be and thus cannot use them at training time for standardization. 

In [ ]:
fstd = np.std(features, axis=0)
fmean = np.mean(features, axis=0)
std_features = (features - fmean) / fstd

In [ ]:
# initialize our paramaters
# since we're changing the features
w = np.random.normal(scale=0.1, size=feature_dim)
b = 0.0


loss_progress = []
eta = 1e-2
batch_size = 32
N = len(labels)  # number of data points
data = (std_features, labels)
# compute how much data fits nicely into a batch
# and drop extra data
new_N = len(labels) // batch_size * batch_size
num_epochs = 3

# the -1 means that numpy will compute
# what that dimension should be
batched_features = std_features[:new_N].reshape((-1, batch_size, feature_dim))
batched_labels = labels[:new_N].reshape((-1, batch_size))
indices = np.arange(new_N // batch_size)

# iterate through the dataset 3 times
for epoch in range(num_epochs):
    # to make it random, we'll iterate over the batches randomly
    np.random.shuffle(indices)
    for i in indices:
        # choose a random set of
        # indices to slice our data
        grad = loss_grad(w, b, (batched_features[i], batched_labels[i]))
        w -= eta * grad[0]
        b -= eta * grad[1]
        # we still compute loss on whole dataset, but not every step
        if i % 50 == 0:
            loss_progress.append(loss_wrapper(w, b, data))

plt.plot(np.arange(len(loss_progress)) * 50, loss_progress)
plt.xlabel("Step")
plt.yscale("log")
plt.ylabel("Loss")
plt.show()

Notice we safely increased our learning rate to 0.01, which is possible because all the features are of similar magnitude. We also could keep training, since we're gaining improvements. 

### Analyzing Model Performance

This is a large topic that we'll explore more, but the first thing we typically examine in supervised learning is a **parity plot**, which shows our predictions vs. our label prediction. What's nice about this plot is that it works no matter what the dimensions of the features are. A perfect fit would fall onto the line at $y = \hat{y}$

In [ ]:
predicted_labels = linear_model(std_features, w, b)

plt.plot([-100, 100], [-100, 100])
plt.scatter(labels, predicted_labels, s=4, alpha=0.7)
plt.xlabel("Measured Solubility $y$")
plt.ylabel("Predicted Solubility $\hat{y}$")
plt.xlim(-13.5, 2)
plt.ylim(-13.5, 2)
plt.show()

Final model assessment can be done with loss, but typically other metrics are also used. In regression, a **correlation coefficient** is typically reported in addition to loss. In our example, this is computed as

In [ ]:
# slice correlation between predict/labels
# from correlation matrix
np.corrcoef(labels, predicted_labels)[0, 1]

In [ ]:
# THIS CELL IS USED TO GENERATE A FIGURE
# AND NOT RELATED TO CHAPTER
# YOU CAN SKIP IT
from myst_nb import glue

glue("corr", np.round(np.corrcoef(labels, predicted_labels)[0, 1], 2))

A correlation coefficient of {glue:}`corr` is OK, but not great.

## Unsupervised Learning

In unsupervised learning, the goal is to predict $\hat{y}$ *without* labels. This seems like an impossible task. How do we judge success? Typically, unsupervised learning can be broken into three categories:

**Clustering**

&nbsp;&nbsp;&nbsp;&nbsp; Here we assume $\{y_i\}$ is a class variable and try to partition our features into these classes. In clustering we are simultaneously learning the definition of the classes (called clusters) and which cluster each feature should be assigned to.

```{margin} Class
In machine learning, a class is a type of label like ``dog`` or ``cat``. Formally,
we have a set of possible labels (e.g., all animals) and each feature vector has one (hard) or a 
probability distribution of classes (soft).
```

**Finding Signal**

&nbsp;&nbsp;&nbsp;&nbsp; $x$ is assumed to be made of two components: noise and signal ($y$). We try to separate the signal $y$ from $x$ and discard noise. Highly-related with **representation learning**, which we'll see later.


**Generative**

&nbsp;&nbsp;&nbsp;&nbsp; Generative methods are methods where we try to learn $P(\vec{x})$ so that we can sample new values of $\vec{x}$. It is analogous to $y$ being probability and we're trying to estimate it. We'll see these more later.



### Clustering

Clustering is historically one of the most well-known and still popular machine learning methods. It's always popular because it can provide new insight from data. Clustering gives class labels where none existed and thus can help find patterns in data. This is also a reason that it has become less popular in chemistry (and most fields): there is no right or wrong answer. Two people doing clustering independently will often arrive at different answers. Nevertheless, it should be a tool you know and can be a good exploration strategy.

```{margin} cluster labels
Clustering comes in many variants and some blur what exactly $y_i$ is. For example, in some clustering methods $y_i$ can include no assignment or $y_i$ is not a single class, but a tree of classes.
```

We'll look at the classic clustering method: k-means. Wikipedia has a [great article](https://en.wikipedia.org/wiki/K-means_clustering) on this classic algorithm, so I won't try to repeat that. To make our clustering actually visible, we'll start by projecting our features into 2 dimensions. This will be covered in representation learning, so don't worry about these steps.

In [ ]:
# get down to 2 dimensions for easy visuals
embedding = sklearn.manifold.Isomap(n_components=2)
# only fit to every 25th point to make it fast
embedding.fit(std_features[::25, :])
reduced_features = embedding.transform(std_features)

We're going to zoom into the middle 99th percentile of the data since some of the points are extremely far away (though that is interesting!). 

In [ ]:
xlow, xhi = np.quantile(reduced_features, [0.005, 0.995], axis=0)

plt.figure(dpi=300)
plt.scatter(
    reduced_features[:, 0],
    reduced_features[:, 1],
    s=4,
    alpha=0.7,
    c=labels,
    edgecolors="none",
)
plt.xlim(xlow[0], xhi[0])
plt.ylim(xlow[1], xhi[1])
cb = plt.colorbar()
cb.set_label("Solubility")
plt.show()

```{margin} Dimensionality Reduction
Reducing $\vec{x}$, your feature vectors to a low
dimensional space. The classic example is PCA, which is a 
linear operator. However, most prefer nonlinear methods 
like [t-SNE](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html).
```



The dimensionality reduction has made our features only 2 dimensions. We can see some structure, especially with the solubility as the coloring. Note in these kind of plots, where we have reduced dimensions in someway, we do not label the axes because they are arbitrary.

Now we cluster. The main challenge in clustering is deciding how many clusters there should be. There are a number of methods out there, but they basically come down to intuition. You, as the chemist, should use some knowledge outside of the data to intuit what is the cluster number. Sounds unscientific? Yeah, that's why clustering is hard.

In [ ]:
# cluster - using whole features
kmeans = sklearn.cluster.KMeans(n_clusters=4, random_state=0)
kmeans.fit(std_features)

Very simple procedure! Now we'll visualize by coloring our data by the class assigned. 

In [ ]:
plt.figure(dpi=300)
point_colors = [f"C{i}" for i in kmeans.labels_]
plt.scatter(
    reduced_features[:, 0],
    reduced_features[:, 1],
    s=4,
    alpha=0.7,
    c=point_colors,
    edgecolors="none",
)
# make legend
legend_elements = [
    plt.matplotlib.patches.Patch(
        facecolor=f"C{i}", edgecolor="none", label=f"Class {i}"
    )
    for i in range(4)
]
plt.legend(handles=legend_elements)
plt.xlim(xlow[0], xhi[0])
plt.ylim(xlow[1], xhi[1])
plt.show()

### Choosing Cluster Number

How do we know we had the correct number? Intuition. There is one tool we can use to help us, called an **elbow plot**. The k-means clusters can be used to compute the mean squared distance from cluster center, basically a version of loss function. However, if we treat cluster number as a trainable parameter we'd find the best fit at the cluster number being equal to number of data points. Not helpful! However, we can see when the slope of this loss becomes approximately constant and assume that those extra clusters are adding no new insight. Let's plot the loss and see what happens. Note we'll be using a subsample of the dataset to save time.

In [ ]:
# make an elbow plot
loss = []
cn = range(2, 15)
for i in cn:
    kmeans = sklearn.cluster.KMeans(n_clusters=i, random_state=0)
    # use every 50th point
    kmeans.fit(std_features[::50])
    # we get score -> opposite of loss
    # so take -
    loss.append(-kmeans.score(std_features[::50]))

plt.plot(cn, loss, "o-")
plt.xlabel("Cluster Number")
plt.ylabel("Loss")
plt.title("Elbow Plot")
plt.show()

Where is the transition? If I squint, maybe at 6? 3? 4? 7? Let's choose 4 because it sounds nice and is plausible based on the data. The last task is to get some insight into what the clusters actually are. We can extract the most centered data points (closest to cluster center) and consider them to be representative of the cluster. 

In [ ]:
# cluster - using whole features
kmeans = sklearn.cluster.KMeans(n_clusters=4, random_state=0)
kmeans.fit(std_features)

cluster_center_idx = []
for c in kmeans.cluster_centers_:
    # find point closest
    i = np.argmin(np.sum((std_features - c) ** 2, axis=1))
    cluster_center_idx.append(i)
cluster_centers = soldata.iloc[cluster_center_idx, :]

legend_text = [f"Class {i}" for i in range(4)]

# now plot them on a grid
cluster_mols = [rdkit.Chem.MolFromInchi(inchi) for inchi in cluster_centers.InChI]
rdkit.Chem.Draw.MolsToGridImage(
    cluster_mols, molsPerRow=2, subImgSize=(400, 400), legends=legend_text
)

So what exactly are these classes? Unclear. We intentionally did not reveal solubility (unsupervised learning) so there is not necessarily any connection with solubility. These classes are more a result of which features were chosen for the dataset. You could make a hypothesis, like class 1 is all negatively charged or class 0 is aliphatic, and investigate. Ultimately though there is no *best* clustering and often unsupervised learning is more about finding insight or patterns and not about producing a highly-accurate model.

The elbow plot method is one of many approaches to selecting cluster number {cite}`pham2005selection`. I prefer it because it's quite clear that you are using intuition. More sophisticated methods sort-of conceal the fact that there is no right or wrong answer in clustering. 



```{note}
This process does not result in a function that predicts solubility. We might try to gain insight about predicting solubility with our predicted classes, but that is not the goal of clustering.
```

## Chapter Summary

* Supervised machine learning is building models that can predict labels $y$ from input features $\vec{x}$.
* Data can be labeled or unlabeled. 
* Models are trained by minimizing loss with stochastic gradient descent.
* Unsupervised learning is building models that can find patterns in data.
* Clustering is unsupervised learning where the model groups the data points into clusters

## Exercises

### Data

1. Using `numpy` reductions `np.amin`, `np.std`, etc. (not `pandas`!), compute the mean, min, max, and standard deviation for each feature across all data points. 

2. Use rdkit to draw the 2 highest molecular weight molecules. Note they look strange. 

### Linear Models

1. Prove that a nonlinear model like $y = \vec{w_1} \cdot \sin\left(\vec{x}\right) + \vec{w_2} \cdot \vec{x} + b$ could be represented as a linear model.

2. Write out the linear model equation in Einstein notation in batched form. **Batched form** means we explicitly have an index indicating batch. For example, the labels will be $y_{bi}$  where $b$ indicates the batch of data and $i$ indicates the individual data point.


### Minimizing Loss

1. We standardized the features, but not the labels. Would standardizing the labels affect our choice of learning rate? Prove your answer.

2. Implement a loss that is mean absolute error, instead of mean squared error. Compute its gradient using `jax`.

2. Using the standardized features, show what effect batch size has on training. Use batch sizes of 1, 8, 32, 256, 1024. Make sure you re-initialize your weights in between each run. Plot the log-loss for each batch size on the same plot. Describe your results.

### Clustering

1. We say that clustering is a type of unsupervised learning and that it predicts the labels. What exactly are the predicted labels in clustering? Write down what the predicted labels might look like for a few data points.

2. In clustering, we predict labels from features. You can still cluster if you have labels, by just pretending they are features. Give two reasons why it would not be a good idea to do clustering in this manner, where we treat the labels as features and try to predict new labels that represent class.

3. On the isomap plot (reduced dimension plot), color the points by which group they fall in (G1, G2, etc.). Is there any relationship between this and the clustering?


## Cited References

```{bibliography}
:style: unsrtalpha
:filter: docname in docnames
```